In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from deap import base, creator, tools, algorithms
import random

In [3]:
# Cargar el dataset de diabetes
data = load_diabetes()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenar el modelo de árbol de decisión con todos los atributos
reg = DecisionTreeRegressor(random_state=42)
reg.fit(X_train, y_train)
# Evaluar el modelo
y_pred = reg.predict(X_test)
mse_all_features = mean_squared_error(y_test, y_pred)
print(f"Error cuadrático medio con todos los atributos: {mse_all_features:.4f}")

Error cuadrático medio con todos los atributos: 5697.7895


In [5]:
# Configuración del Algoritmo Genético
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Ahora minimizamos el error (mse)
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()

# Los primeros 'X_train.shape[1]' valores son las características, los siguientes son max_depth y min_samples_split
toolbox.register("attr_bool", lambda: random.randint(0, 1))  # Para las características (0 o 1)
toolbox.register("attr_max_depth", lambda: random.randint(2, 20))  # Para max_depth (profundidad entre 2 y 20)
toolbox.register("attr_min_samples_split", lambda: random.randint(2, 20))  # Para min_samples_split (2 a 20)

# Crear individuo con 10 características y los hiperparámetros max_depth y min_samples_split
def create_individual():
    features = [toolbox.attr_bool() for _ in range(X_train.shape[1])]  # 10 valores binarios para diabetes
    max_depth = toolbox.attr_max_depth()  # Profundidad máxima del árbol
    min_samples_split = toolbox.attr_min_samples_split()  # Muestras mínimas para dividir
    individual = creator.Individual(features + [max_depth, min_samples_split])  # Convertir en un objeto Individual
    return individual

toolbox.register("individual", create_individual)

# Crear la población
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Función de evaluación
def evaluate(individual):
    # Las características seleccionadas son desde el primer elemento hasta el antepenúltimo
    selected_features = [i for i in range(len(individual) - 2) if individual[i] == 1]  # Ignorar los últimos dos valores
    max_depth = int(individual[-2])  # Penúltimo valor es max_depth
    min_samples_split = int(individual[-1])  # Último valor es min_samples_split

    if len(selected_features) == 0:
        return float('inf'),  # Evitar selecciones vacías devolviendo un error muy alto

    # Seleccionar las características correctas del conjunto de entrenamiento y prueba
    X_train_selected = X_train.iloc[:, selected_features]
    X_test_selected = X_test.iloc[:, selected_features]

    # Crear el regressor con max_depth y min_samples_split del individuo
    reg = DecisionTreeRegressor(random_state=42, max_depth=max_depth, min_samples_split=min_samples_split)
    reg.fit(X_train_selected, y_train)
    y_pred = reg.predict(X_test_selected)

    return mean_squared_error(y_test, y_pred),

# Mutación de características, max_depth y min_samples_split
def custom_mutate(individual, indpb):
    # Mutar las características binarias (0/1)
    for i in range(len(individual) - 2):  # Características
        if random.random() < indpb:
            individual[i] = 1 if individual[i] == 0 else 0

    # Mutar max_depth
    if random.random() < indpb:
        individual[-2] = random.randint(2, 20)  # Mutar max_depth a un nuevo valor
    
    # Mutar min_samples_split
    if random.random() < indpb:
        individual[-1] = random.randint(2, 20)  # Mutar min_samples_split a un nuevo valor
    
    return individual,

# Operadores genéticos
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", custom_mutate, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Configuración del algoritmo genético
population = toolbox.population(n=100)
ngen = 100

# Ejecutar el algoritmo genético
result_population, logbook = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=ngen, verbose=False)

# Seleccionar el mejor individuo
best_individual = tools.selBest(result_population, k=1)[0]

# Extraer las características seleccionadas, max_depth, y min_samples_split por separado
selected_features = [i for i in range(len(best_individual) - 2) if best_individual[i] == 1]  # Características
max_depth = best_individual[-2]  # Penúltimo valor es max_depth
min_samples_split = best_individual[-1]  # Último valor es min_samples_split

print(f"Mejores características seleccionadas: {selected_features}")
print(f"Max depth seleccionado: {max_depth}")
print(f"Min samples split seleccionado: {min_samples_split}")

best_fit = best_individual.fitness.values[0]  # El fitness es una tupla, por lo que accedes al primer valor
print(f"Best fitness (mse): {best_fit}")

c:\Users\usuario\anaconda3\envs\env_101\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\usuario\anaconda3\envs\env_101\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Mejores características seleccionadas: [0, 1, 4, 7, 8, 9]
Max depth seleccionado: 2
Min samples split seleccionado: 13
Best fitness (mse): 3035.7523569750565
